In [ ]:
# --- STATIC SIMULATION FOR THRESHOLD EXCEEDANCE ---
# Works with sklearn .pkl or Keras .h5 models
# Assumes you already have a dataframe `df` with weather/time/polarization features
# and a raw fidelity column for ground-truth lookahead.

import os
import math
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

# If you plan to load .pkl models:
import joblib

# If you plan to load .h5 models:
try:
    from tensorflow.keras.models import load_model as keras_load_model
except Exception:
    keras_load_model = None  # only needed if you actually load a .h5

# =========================
# CONFIG (edit these)
# =========================
MODEL_PATH = "model.pkl"     # e.g., "xgb.pkl", "svr.pkl", or "nn.h5"
FEATS      = [               # exact feature list used in training, in order
    # e.g., "temp_x", "temp_y", "tod_sin", "tod_cos", "wind_u_x", ...
    # Fill with your training-time feature names:
]
FID_COL    = "Fidelity"      # raw polarization fidelity column for ground-truth lookahead
TARGET_KIND = "rstdev"       # one of: "rstdev", "rextdiff", "absdiff"
THRESH     = 0.10            # absolute fidelity-drift tolerance (example)
HORIZON_H  = 300             # how far ahead to look (in samples) for a true exceedance
RSTDEV_WINDOW = 300          # model’s intended window length for std (if relevant)
GAUSS_P_EXCEED_THRESH = 0.05 # for rstdev: threshold on predicted exceedance percent -> flag

# Optional: standardize X for Keras models if they were trained on standardized inputs
APPLY_SCALER = False
SCALER_MEAN = None  # set np.array of means if you saved them
SCALER_STD  = None  # set np.array of stds   if you saved them

# Optional: post-processing for model outputs (if you trained on a scaled y)
Y_MEAN = None
Y_STD  = None

# =========================
# UTILITIES
# =========================
def load_any_model(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".pkl":
        return joblib.load(path), "sklearn"
    elif ext == ".h5":
        if keras_load_model is None:
            raise RuntimeError("TensorFlow/Keras not available to load .h5 models.")
        return keras_load_model(path), "keras"
    else:
        raise ValueError(f"Unsupported model extension: {ext}")

def predict_any(model, kind, X):
    if kind == "sklearn":
        yhat = model.predict(X)
    else:
        # Keras expects 2D (N, F). If your NN expects sequences, adapt here.
        yhat = model.predict(X, verbose=0).ravel()
    return np.asarray(yhat, dtype=float)

def maybe_unscale_y(yhat):
    if (Y_MEAN is not None) and (Y_STD is not None) and (Y_STD != 0):
        return yhat * float(Y_STD) + float(Y_MEAN)
    return yhat

def gaussian_exceed_prob(sigma, tol):
    """Two-sided exceedance P(|X|>tol) for N(0, sigma^2)."""
    if sigma <= 0:
        return 0.0
    z = tol / sigma
    # 1 - Phi(z) using error function
    tail = 0.5 * math.erfc(z / math.sqrt(2.0))
    return 2.0 * tail

def chebyshev_upper_bound(sigma, tol):
    """Chebyshev upper bound on P(|X - mu| >= tol) <= (sigma/tol)^2."""
    if sigma <= 0:
        return 0.0
    r = sigma / tol
    return float(min(1.0, r * r))

def ground_truth_exceed_in_horizon(fid, idx, tol, H):
    """
    True label for a given index: does |fidelity[t+h] - fidelity[t]| exceed tol at
    any step 1..H? (You can change the reference if needed.)
    """
    ref = fid[idx]
    end = min(len(fid), idx + H + 1)
    if end <= idx + 1:
        return 0
    segment = fid[idx+1:end]
    return int(np.any(np.abs(segment - ref) > tol))

def split_train_val_test(n, tr=0.70, va=0.15):
    n_tr = int(n * tr)
    n_va = int(n * va)
    n_te = n - n_tr - n_va
    return slice(0, n_tr), slice(n_tr, n_tr + n_va), slice(n_tr + n_va, n)

# =========================
# MAIN SIMULATION
# =========================
# 0) Basic checks
assert isinstance(df, pd.DataFrame), "Expected a DataFrame named `df` to be present."
for c in FEATS + [FID_COL]:
    if c not in df.columns:
        raise KeyError(f"Column `{c}` not in df.")

# 1) Prep features/labels (test split only)
X_all = df[FEATS].copy()
fid   = df[FID_COL].astype(float).to_numpy()

idx_tr, idx_va, idx_te = split_train_val_test(len(df))
X_te = X_all.iloc[idx_te].reset_index(drop=True)
fid_te = fid[idx_te]

if APPLY_SCALER:
    if SCALER_MEAN is None or SCALER_STD is None:
        raise RuntimeError("APPLY_SCALER=True but SCALER_MEAN/SCALER_STD not provided.")
    X_te = (X_te.to_numpy(dtype=float) - SCALER_MEAN) / SCALER_STD
else:
    X_te = X_te.to_numpy(dtype=float)

# 2) Load model and predict on test features
model, model_kind = load_any_model(MODEL_PATH)
yhat_te = predict_any(model, model_kind, X_te)
yhat_te = maybe_unscale_y(yhat_te)

# 3) Interpretation layer → predicted exceed flag (and extras for rstdev)
pred_flag = np.zeros(len(yhat_te), dtype=int)
gauss_p   = np.full(len(yhat_te), np.nan)
cheb_p    = np.full(len(yhat_te), np.nan)

if TARGET_KIND == "rstdev":
    # yhat = predicted sigma over a window; convert to exceedance probabilities
    for i, s in enumerate(yhat_te):
        gp = gaussian_exceed_prob(s, THRESH)
        cb = chebyshev_upper_bound(s, THRESH)
        gauss_p[i] = gp
        cheb_p[i]  = cb
        pred_flag[i] = int(gp >= GAUSS_P_EXCEED_THRESH)

elif TARGET_KIND == "rextdiff":
    # yhat = predicted peak-to-peak over a window
    pred_flag = (yhat_te > THRESH).astype(int)

elif TARGET_KIND == "absdiff":
    # yhat = predicted absolute first-difference at the next step
    pred_flag = (yhat_te > THRESH).astype(int)

else:
    raise ValueError("TARGET_KIND must be one of {'rstdev','rextdiff','absdiff'}.")

# 4) Ground-truth labels via lookahead in the *actual fidelity*
true_flag = np.zeros(len(fid_te), dtype=int)
for i in range(len(fid_te)):
    true_flag[i] = ground_truth_exceed_in_horizon(fid_te, i, THRESH, HORIZON_H)

# 5) Align lengths (if model produced fewer predictions for any reason)
m = min(len(pred_flag), len(true_flag))
pred_flag = pred_flag[:m]
true_flag = true_flag[:m]
if TARGET_KIND == "rstdev":
    gauss_p = gauss_p[:m]
    cheb_p  = cheb_p[:m]

# 6) Metrics
cm = confusion_matrix(true_flag, pred_flag, labels=[0,1])
tn, fp, fn, tp = cm.ravel()
print("=== Confusion Matrix (rows=true, cols=pred) ===")
print(pd.DataFrame(cm, index=["True 0","True 1"], columns=["Pred 0","Pred 1"]))
print("\n=== Classification Report ===")
print(classification_report(true_flag, pred_flag, digits=3))

if TARGET_KIND == "rstdev":
    print(f"\nAvg predicted Gaussian exceedance P(|X|>{THRESH}): {np.nanmean(gauss_p):.4f}")
    print(f"Avg predicted Chebyshev upper bound:            {np.nanmean(cheb_p):.4f}")

# 7) Optional: regression diagnostics of raw predictions vs simple targets
#    (only meaningful if your model target is comparable to a numeric truth)
try:
    rmse = lambda a,b: float(np.sqrt(mean_squared_error(a,b)))
    print("\n=== Regression Diagnostics (if applicable) ===")
    print(f"Sample size used: {m}")
    print("Note: These RMSE/R^2 values are meaningful only if yhat is directly comparable to a numeric truth.")
except Exception:
    pass


In [ ]:
import math
import pickle
from collections import deque

import numpy as np
import pandas as pd

class StaticSim:
    """
    Streaming/static rollout simulator.
    - step(...) consumes ONE observation (row) at a time.
    - Maintains internal state (buffers, accumulators).
    - Main program handles visualization/printing.
    """

    def __init__(self, args):
        # Required config
        self.pred_method = args["pred_method"]      # "rstdev", "rexd", "fder", "fder_sder"
        self.target      = args.get("pred_targ", "")# informational
        self.thresh      = float(args["thresh"])    # e.g., 0.1
        self.win_size    = int(args["tol"])         # window length in steps (for buffers)
        self.sr          = float(args["sr"])        # sample rate in seconds per step (dt)
        self.dt          = self.sr                  # alias

        # For rstdev: probability threshold
        # If you want "flag when P(|dev|>thresh) >= p0", set p0 here.
        # Your old gauss_thresh = thresh/2 is dimensionally weird; better is prob threshold.
        self.p_exceed_thresh = float(args.get("p_exceed_thresh", 0.2))  # e.g., 20%

        # For derivative horizon checks
        self.horizon_steps = int(args.get("horizon_steps", 5))          # e.g., 5 steps ahead
        self.H = self.horizon_steps * self.dt

        # For total-variation accumulation (streaming)
        self.use_tv = bool(args.get("use_total_variation", True))
        self.use_horizon = bool(args.get("use_horizon_forecast", True))

        # If using EMA for variation instead of strict window sum:
        self.tv_tau_sec = float(args.get("tv_tau_sec", self.win_size * self.dt))
        self.tv_alpha = min(1.0, self.dt / self.tv_tau_sec) if self.tv_tau_sec > 0 else 1.0

        # Scaling toggles
        self.output_scale = bool(args.get("output_scaled", False))
        self.input_scale  = bool(args.get("input_scaled",  False))

        # Column names
        self.posix_feat = args.get("posix_feat_name", "current_time")
        self.fid_feat   = args.get("fid_feat_name", "Fidelity")

        # Feature columns used by the model (important for streaming)
        self.feature_cols = args.get("feature_cols", None)  # list[str]
        if self.feature_cols is None:
            raise ValueError("args['feature_cols'] must be provided for streaming inference.")

        # Load model(s)
        self.load_model(args["model_path"])
        self.model_d2 = None
        if self.pred_method in ("fder_sder", "d1_d2", "fder+sder"):
            self.load_model_d2(args["model_path_d2"])

        # Data optional (you can still use run(df))
        self.df = None
        if "data_path" in args:
            self.load_data(args["data_path"], self.posix_feat, args.get("test_start_posix", -np.inf))

        # Scaling params (optional)
        if self.output_scale:
            self.output_scale_mean = float(args["output_scale_mean"])
            self.output_scale_stdev = float(args["output_scale_stdev"])
        if self.input_scale:
            self.input_scale_mean = np.array(args["input_scale_mean"], dtype=float)
            self.input_scale_stdev = np.array(args["input_scale_stdev"], dtype=float)

        # Internal state
        self.reset()

    # ---------- I/O ----------
    def load_model(self, model_path):
        # TODO: detect sklearn vs keras; for now keep pickle
        with open(model_path, "rb") as f:
            self.model = pickle.load(f)

    def load_model_d2(self, model_path_d2):
        with open(model_path_d2, "rb") as f:
            self.model_d2 = pickle.load(f)

    def load_data(self, data_path, posix_name, posix_val):
        df = pd.read_csv(data_path)
        self.df = df[df[posix_name] >= posix_val].reset_index(drop=True)

    # ---------- Scaling ----------
    def _scale_inputs(self, x: np.ndarray) -> np.ndarray:
        if not self.input_scale:
            return x
        return (x - self.input_scale_mean) / self.input_scale_stdev

    def _unscale_output(self, yhat: float) -> float:
        if not self.output_scale:
            return float(yhat)
        return float(yhat) * self.output_scale_stdev + self.output_scale_mean

    # ---------- State ----------
    def reset(self):
        # buffers for diagnostics / accumulation
        self.t_buf = deque(maxlen=self.win_size)
        self.f_buf = deque(maxlen=self.win_size)

        self.yhat_buf = deque(maxlen=self.win_size)   # primary prediction history
        self.d2_buf   = deque(maxlen=self.win_size)   # if used

        # For total-variation tracking from |d1|
        self.tv_ema = 0.0
        self.tv_window = deque(maxlen=self.win_size)  # store per-step contributions |d1|*dt

        # counters
        self.k = 0

    # ---------- Core step ----------
    def step(self, obs) -> dict:
        """
        obs: dict-like or pd.Series with at least:
            - posix time column (self.posix_feat)
            - Fidelity column (self.fid_feat) [optional for model, useful for logging]
            - all model feature columns (self.feature_cols)
        Returns a dict with predictions and flags for this step.
        """
        if isinstance(obs, pd.Series):
            obs = obs.to_dict()

        t = obs.get(self.posix_feat, None)
        f = obs.get(self.fid_feat, None)

        # Build feature vector for model
        x = np.array([obs[c] for c in self.feature_cols], dtype=float)
        x = self._scale_inputs(x).reshape(1, -1)

        # Predict primary target
        yhat = float(self.model.predict(x).reshape(-1)[0])
        yhat = self._unscale_output(yhat)

        # Optional second-derivative prediction (absolute)
        yhat_d2 = None
        if self.model_d2 is not None:
            yhat_d2 = float(self.model_d2.predict(x).reshape(-1)[0])
            # you may need its own unscale params if trained differently

        # Update internal buffers
        self.t_buf.append(t)
        self.f_buf.append(f)
        self.yhat_buf.append(yhat)
        if yhat_d2 is not None:
            self.d2_buf.append(yhat_d2)

        # Compute flag + additional metrics depending on method
        out = {
            "step": self.k,
            "t": t,
            "Fidelity": f,
            "yhat": yhat,
        }
        if yhat_d2 is not None:
            out["yhat_d2"] = yhat_d2

        flag_info = self.update_flags_stream(yhat, yhat_d2=yhat_d2)
        out.update(flag_info)

        self.k += 1
        return out

    # ---------- Streaming flags ----------
    def update_flags_stream(self, yhat, yhat_d2=None) -> dict:
        method_name = f"_flags_{self.pred_method}"
        if not hasattr(self, method_name):
            raise ValueError(f"No update method exists for {method_name}")
        return getattr(self, method_name)(yhat, yhat_d2=yhat_d2)

    # --- rstdev: turn σ into exceedance probability ---
    def _flags_rstdev(self, s, yhat_d2=None) -> dict:
        s = float(s)
        if s <= 0:
            p_gauss = 0.0
            p_cheby = 0.0
        else:
            z = self.thresh / s
            tail = 0.5 * math.erfc(z / math.sqrt(2.0))     # 1 - Phi(z)
            p_gauss = 2.0 * tail                           # P(|X|>T) for N(0,s^2)
            r = s / self.thresh
            p_cheby = float(min(1.0, r * r))               # Chebyshev upper bound

        flag = int(p_gauss >= self.p_exceed_thresh)
        return {
            "flag": flag,
            "p_exceed_gauss": p_gauss,
            "p_exceed_cheby_ub": p_cheby,
        }

    # --- rexd: direct excursion ---
    def _flags_rexd(self, r, yhat_d2=None) -> dict:
        r = float(r)
        return {"flag": int(r > self.thresh), "deltaF_est": r}

    # --- fder: |dF/dt| (absolute) ---
    def _flags_fder(self, abs_d1, yhat_d2=None) -> dict:
        abs_d1 = float(abs_d1)

        # 1) Horizon forecast upper bound
        delta_h = abs_d1 * self.H if self.use_horizon else np.nan
        flag_h = int(delta_h > self.thresh) if self.use_horizon else 0

        # 2) Total-variation accumulation (windowed or EMA)
        tv_contrib = abs_d1 * self.dt
        self.tv_window.append(tv_contrib)
        tv_win = float(sum(self.tv_window))  # in Fidelity units

        # EMA version (smoother, effectively windowed by tau)
        self.tv_ema = (1.0 - self.tv_alpha) * self.tv_ema + self.tv_contrib
        tv_used = self.tv_ema if self.use_tv else np.nan
        flag_tv = int(tv_used > self.thresh) if self.use_tv else 0

        # Combine flags (you can choose OR / AND)
        flag = int((flag_h == 1) or (flag_tv == 1))

        return {
            "flag": flag,
            "deltaF_horizon_ub": delta_h,
            "tv_window_sum": tv_win,
            "tv_ema": self.tv_ema,
            "flag_horizon": flag_h,
            "flag_tv": flag_tv,
        }

    # --- fder_sder: |d1| + |d2| used for an upper-bound forecast ---
    def _flags_fder_sder(self, abs_d1, yhat_d2=None) -> dict:
        if yhat_d2 is None:
            raise ValueError("fder_sder requires yhat_d2 prediction/model.")

        abs_d1 = float(abs_d1)
        abs_d2 = float(yhat_d2)

        # Upper bound forecast over horizon H: ΔF <= |d1|H + 0.5|d2|H^2
        delta_h = abs_d1 * self.H + 0.5 * abs_d2 * (self.H ** 2)
        flag_h = int(delta_h > self.thresh)

        # Also keep total variation from |d1| (still useful)
        tv_contrib = abs_d1 * self.dt
        self.tv_window.append(tv_contrib)
        self.tv_ema = (1.0 - self.tv_alpha) * self.tv_ema + self.tv_contrib
        flag_tv = int(self.tv_ema > self.thresh) if self.use_tv else 0

        flag = int((flag_h == 1) or (flag_tv == 1))

        return {
            "flag": flag,
            "deltaF_horizon_ub": delta_h,
            "tv_ema": self.tv_ema,
            "flag_horizon": flag_h,
            "flag_tv": flag_tv,
        }

    # ---------- Convenience: batch run ----------
    def run_df(self, df: pd.DataFrame) -> pd.DataFrame:
        self.reset()
        rows = []
        for _, row in df.iterrows():
            rows.append(self.step(row))
        return pd.DataFrame(rows)

    def run(self) -> pd.DataFrame:
        if self.df is None:
            raise ValueError("No data loaded. Use run_df(df) or provide data_path in args.")
        return self.run_df(self.df)


In [ ]:
model_path: ./model.pkl
data_path: ./data.csv
pred_targ: RSTDEV_Fidelity_300_SAVGOL
features:
 - x
pred_method: rstdev # rstdev, rexd, fder
test_start_posix: 2
posix_feat_name: current_time
tol: 300 # aggregation window size (observations)
sr: 30 # sample rate (seconds)
thresh: 0.05
output_scaled: False
output_scale_mean: 0
output_scale_stdev: 0
input_scaled: False
input_scale_mean: 0
input_scale_stdev: 0
plot: False
poincare: False

p_exceed_thresh: 0.2     # flag if P(|dev| > thresh) >= 20%
horizon_steps: 5         # predict threshold crossing within next 5 steps
tv_tau_sec: 900          # EMA timescale for total-variation accumulation (optional)
use_total_variation: True
use_horizon_forecast: True

In [ ]:
import yaml

def load_config(yaml_path: str) -> dict:
    with open(yaml_path, "r") as f:
        cfg = yaml.safe_load(f)

    # Normalize key names used by the simulator
    # Your YAML uses `features`, sim uses `feature_cols`
    cfg["feature_cols"] = cfg.pop("features", [])

    # Optional: if you want derivative horizons / stdev probability thresholds
    # cfg.setdefault("horizon_steps", 5)
    # cfg.setdefault("p_exceed_thresh", 0.2)

    return cfg

Main python file looks like this:

In [ ]:
from .StaticSim import StaticSim
import yaml
import sys

def main(args):

    sim = StaticSim(args)

if __name__ == '__main__':
    if len(sys.argv) < 2:
        print("Please set config yaml path")
        exit(1)
    with open(sys.argv[1]) as f:
        args = yaml.safe_load(f)

    main(args)

In [ ]:
args = yaml.safe_load(open("./config.yaml"))
sim = main(args)

df = pd.read_csv(args["data_path"])
df = df[df[args["posix_feat_name"]] >= args["test_start_posix"]].reset_index(drop=True)

sim.reset()

outs = []
for _, row in df.iterrows():
    outs.append(sim.step(row))

df_sim = pd.DataFrame(outs)
df_sim.head()